
# Summary: Multilinear Regression with SVD and Confidence Intervals

## 1. Singular Value Decomposition (SVD) for Regression

Given a system $Ax = b$ where:
- $A$: Design matrix (n_samples × n_features)
- $b$: Target vector (n_samples × 1)
- $x$: Coefficient vector to find (n_features × 1)

### SVD Decomposition
SVD factorizes $A$ into:
$A = U \Sigma V^T$
where:
- $U$: Left singular vectors (orthonormal basis for column space of A)
- $\Sigma$: Diagonal matrix of singular values ($\sigma_1 \geq \sigma_2 \geq ... \geq \sigma_r > 0$)
- $V$: Right singular vectors (orthonormal basis for row space of A)

### Solving for x
The least-squares solution is:
$x = V \Sigma^+ U^T b$
where $\Sigma^+$ is the pseudoinverse of $\Sigma$ (reciprocal of non-zero $\sigma_i$).

---

## 2. Key Concepts

### Root Mean Square Error (RMSE)
Measures model prediction error:
$\text{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^n (b_i - \hat{b}_i)^2}$
where $\hat{b} = Ax$ are the predicted values.

### Leverage ($h_i$)
Quantifies how much influence a data point has on the model fit:
$h_i = H_{ii} \quad \text{(diagonal of the hat matrix } H = U U^T)$
- **Range**: $0 \leq h_i \leq 1$
- **Interpretation**:
  - $h_i \approx 0$: Low influence
  - $h_i \approx 1$: High influence (outlier in feature space)

### Standard Error of Prediction ($SE_{pred}$)
Uncertainty for a prediction at point $A_i$:
$SE_{\text{pred}} = \text{RMSE} \times \sqrt{1 + h_i}$
- Accounts for both residual error (RMSE) and leverage.

---

## 3. Handling New Data Outside Training Set

### Computing Leverage for New Data
For a new observation $A_{\text{new}}$ (1 × m vector):

1. **Project onto SVD basis**:
   $h_{\text{new}} = A_{\text{new}} (A^T A)^{-1} A_{\text{new}}^T$

2. **Using SVD components** (more stable):
   $h_{\text{new}} = \|U^T A_{\text{new}}^T\|^2$

### Python Implementation
```python
# For single new observation
A_new = np.array([[...]])  # 1 × m vector
h_new = (U.T @ A_new.T).ravel() @ (U.T @ A_new.T).ravel()

# For multiple new observations (matrix A_new)
h_new = np.sum((U.T @ A_new.T)**2, axis=0)
```

### Prediction Uncertainty
- Standard Error:
  $SE_{\text{new}} = \text{RMSE} \times \sqrt{1 + h_{\text{new}}}$
- 95% Prediction Interval:
  $\hat{b}_{\text{new}} \pm t_{n-m, 0.975} \times SE_{\text{new}}$

### Interpretation
- $h_{\text{new}} \approx 0$: Similar to training data → lower uncertainty
- $h_{\text{new}} \gg 0$: Far from training data → higher uncertainty
- When $h_{\text{new}} > \frac{2m}{n}$, the point is considered high-leverage

---

## 4. Confidence Intervals (CI)

### 95% CI for Mean Prediction
$\hat{b}_i \pm t_{n-m, 0.975} \cdot \text{RMSE} \sqrt{h_i}$
where:
- $t_{n-m, 0.975}$: Critical value from t-distribution with $n - m$ degrees of freedom
- Covers the **true mean response** with 95% confidence.

### 95% Prediction Interval (PI)
For individual future observations:
$\hat{b}_i \pm t_{n-m, 0.975} \cdot \text{RMSE} \sqrt{1 + h_i}$
- Wider than CI (accounts for both model uncertainty and noise).

---

## 5. Practical Steps

1. **Compute SVD** of A:
   ```python
   U, S, Vt = np.linalg.svd(A, full_matrices=False)
   ```

2. **Calculate coefficients x**:
   ```python
   x = Vt.T @ np.diag(1/S) @ U.T @ b
   ```

3. **Compute leverage**:
   ```python
   h_i = np.diag(U @ U.T)
   ```

4. **RMSE**:
   ```python
   residuals = b - A @ x
   RMSE = np.sqrt(np.mean(residuals**2))
   ```

5. **Confidence/Prediction Intervals**:
   ```python
   SE_pred = RMSE * np.sqrt(1 + h_i)  # For PI
   CI = t.ppf(0.975, n - m) * RMSE * np.sqrt(h_i)  # For mean CI
   ```

---

## 6. Interpretation

- **High-leverage points** ($h_i \to 1$) inflate $SE_{pred}$, reflecting higher uncertainty.
- **New data warnings**:
  - When $h_{\text{new}} > \frac{2m}{n}$, predictions are extrapolations
  - Always report $h_{\text{new}}$ alongside predictions
- **95% CI**: "Where the regression line is likely to be."
- **95% PI**: "Where future data points are likely to fall."

This approach is numerically stable and handles rank-deficient matrices gracefully.
```